<a href="https://colab.research.google.com/github/sallywang147/llmfz/blob/main/FuzzTargets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet accelerate
!pip install --quiet bitsandbytes
!pip install --quiet datasets
!pip install --quiet peft
!pip install --quiet xlrd
!pip install --quiet simplejson
!pip install --quiet rouge_score
!pip install --quiet fire
!pip install --quiet openai
!pip install --quiet transformers
!pip install --quiet torch
!pip install --quiet sentencepiece
!pip install --quiet tokenizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [6]:
import os
import sys
import argparse
import json
import random
import torch
import transformers
from datasets import Dataset, load_dataset
from accelerate import dispatch_model, infer_auto_device_map
from accelerate.utils import get_balanced_memory
from torch.cuda.amp import autocast
from transformers import (
    LlamaForCausalLM,
    LlamaTokenizer,
    GenerationConfig,
    BitsAndBytesConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq,
)
from peft import (
    prepare_model_for_int8_training,
    LoraConfig,
    PeftConfig,
    get_peft_model,
    PeftModel,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
)

In [47]:
def generate_text(prompt):
  DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
  peft_model_id = "sallywww/LLaMA_oneStep_fuzzTragets"
  model = LlamaForCausalLM.from_pretrained(
            "baffo32/decapoda-research-llama-7B-hf",
            load_in_8bit=True,
            torch_dtype=torch.float16,
            device_map= "auto", #comment this line out if encountering cuda OOM or wrong distribution
                                #across GPU and CPU.
        )
  tokenizer = LlamaTokenizer.from_pretrained(
            "baffo32/decapoda-research-llama-7B-hf",
        )
  peftmodel = PeftModel.from_pretrained(model, peft_model_id, torch_dtype=torch.float16)
  inputs = tokenizer(prompt, return_tensors="pt")
  model.config.use_cache = False
  old_state_dict = model.state_dict
  model.state_dict = (lambda self, *_, **__: get_peft_model_state_dict(self, old_state_dict())).__get__(
        model, type(model)
    )
  if torch.__version__ >= "2" and sys.platform != "win32":
        print("compiling the model")
        model = torch.compile(model)
       # input_ids = inputs["input_ids"].to(model.device)
  generation_config = GenerationConfig(
        temperature=0.1,
        top_p=0.75,
        repetition_penalty=1.1,
    )
  with torch.no_grad():
      outputs = peftmodel.generate(input_ids=inputs["input_ids"].cuda(), generation_config=generation_config)
      output_text = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
      print(output_text)
  return output_text

In [7]:
def generate_and_tokenize_prompt(func_signature):
    full_prompt =f"""You are a security testing engineer who wants to write a C++ program to execute all lines in a given function by defining and initializing its parameters in a suitable way before fuzzing the function through <code>LLVMFuzzerTestOneInput</code>.
    Carefully study the function signature and its parameters, then generate a fuzz targets by the following instructions. YOU MUST call the function to fuzz in the solution.
    Try as many variations of possible inputs as possible. Do not use a random number generator such as <code>rand()</code>.
    Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>.
    All variables used MUST be declared and initialized. Carefully make sure that the variable and argument types in your code match and compiles successfully. Add type casts to make types match.
    Do not create new variables with the same names as existing variables. If you write code using <code>goto</code>, you MUST MUST also declare all variables BEFORE the <code>goto</code>. Never introduce new variables after the <code>goto</code>.
    It is important that the provided solution compiles and actually calls the function specified by the function signature:

    ### Input:
    {func_signature}

     ### Response:
    """
    return full_prompt

In [8]:
harfbuzz_sigs = ["struct hb_face_t * hb_subset_preprocess(struct hb_face_t * source)",
                 "void hb_ot_shape_glyphs_closure(struct hb_font_t * font, struct hb_buffer_t * buffer, struct hb_feature_t * features, int num_features, struct hb_set_t * glyphs)",
                 "int hb_shape_justify(struct hb_font_t * font, struct hb_buffer_t * buffer, struct hb_glyph_extents_t * features, int num_features, char ** shaper_list, float min_target_advance, float max_target_advance, float * advance, int * var_tag, float * var_value)",
                 "double solve_itp<hb_shape_justify::$_3>(anon * f, double a, double b, double epsilon, double min_y, double max_y, double * ya, double * yb, double * y)",
                 "double hb_shape_justify::$_3::operator(anon * this, double x)",
                 ]
harfbuzz_count = 1

In [ ]:
#!pip install git+https://github.com/huggingface/transformers

In [48]:
import os
if os.path.isdir("../finetuned_results") is False:
  os.mkdir("../finetuned_results")
if os.path.isdir("../finetuned_results/output_harfbuzz") is False:
  os.mkdir("../finetuned_results/output_harfbuzz")
for i in harfbuzz_sigs:
  raw_target = open(f"../finetuned_results/output_harfbuzz/0{harfbuzz_count}.rawoutput", "w+")
  prompt = generate_and_tokenize_prompt(i)
  generated_target = generate_text(prompt)
  raw_target.write(generated_target)
  print(generated_target)
  raw_target.close()
  harfbuzz_count += 1




Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

compiling the model
You are a security testing engineer who wants to write a C++ program to execute all lines in a given function by defining and initializing its parameters in a suitable way before fuzzing the function through <code>LLVMFuzzerTestOneInput</code>.
    Carefully study the function signature and its parameters, then generate a fuzz targets by the following instructions. YOU MUST call the function to fuzz in the solution.
    Try as many variations of possible inputs as possible. Do not use a random number generator such as <code>rand()</code>.
    Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>.
    All variables used MUST be declared and initialized. Carefully make sure that the variable and argument types in your code match and compiles successfully. Add type casts to make types match.
    Do not create new variables with the same names as existing variables. If you write code using <code>goto</code>, y

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

compiling the model
You are a security testing engineer who wants to write a C++ program to execute all lines in a given function by defining and initializing its parameters in a suitable way before fuzzing the function through <code>LLVMFuzzerTestOneInput</code>.
    Carefully study the function signature and its parameters, then generate a fuzz targets by the following instructions. YOU MUST call the function to fuzz in the solution.
    Try as many variations of possible inputs as possible. Do not use a random number generator such as <code>rand()</code>.
    Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>.
    All variables used MUST be declared and initialized. Carefully make sure that the variable and argument types in your code match and compiles successfully. Add type casts to make types match.
    Do not create new variables with the same names as existing variables. If you write code using <code>goto</code>, y

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

...

You are a security testing engineer who wants to write a C++ program to execute all lines in a given function by defining and initializing its parameters in a suitable way before fuzzing the function through <code>LLVMFuzzerTestOneInput</code>.
    Carefully study the function signature and its parameters, then generate a fuzz targets by the following instructions. YOU MUST call the function to fuzz in the solution.
    Try as many variations of possible inputs as possible. Do not use a random number generator such as <code>rand()</code>.
    Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>.
    All variables used MUST be declared and initialized. Carefully make sure that the variable and argument types in your code match and compiles successfully. Add type casts to make types match.
    Do not create new variables with the same names as existing variables. If you write code using <code>goto</code>, you MUST MUST also de

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

compiling the model


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 313, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  encoder = getattr(self, "encoder", None)


You are a security testing engineer who wants to write a C++ program to execute all lines in a given function by defining and initializing its parameters in a suitable way before fuzzing the function through <code>LLVMFuzzerTestOneInput</code>.
    Carefully study the function signature and its parameters, then generate a fuzz targets by the following instructions. YOU MUST call the function to fuzz in the solution.
    Try as many variations of possible inputs as possible. Do not use a random number generator such as <code>rand()</code>.
    Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>.
    All variables used MUST be declared and initialized. Carefully make sure that the variable and argument types in your code match and compiles successfully. Add type casts to make types match.
    Do not create new variables with the same names as existing variables. If you write code using <code>goto</code>, you MUST MUST also de

In [18]:
def generate_rawllama_text(prompt):
  DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
  model = LlamaForCausalLM.from_pretrained(
            "baffo32/decapoda-research-llama-7B-hf",
        )
  tokenizer = LlamaTokenizer.from_pretrained(
            "baffo32/decapoda-research-llama-7B-hf",
        )
  inputs = tokenizer(prompt, return_tensors="pt")
  model.config.use_cache = False
       # input_ids = inputs["input_ids"].to(model.device)
  generation_config = GenerationConfig(
        temperature=0.1,
        top_p=0.75,
        repetition_penalty=1.1,
    )
  with torch.no_grad():
      outputs = model.generate(input_ids=inputs["input_ids"].to(model.device), generation_config=generation_config)
      output_text = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
      print(output_text)
  return output_text

In [19]:
if os.path.isdir("../finetuned_results") is False:
  os.mkdir("../finetuned_results")
if os.path.isdir("../finetuned_results/output_harfbuzz") is False:
  os.mkdir("../finetuned_results/output_harfbuzz")
for i in harfbuzz_sigs:
  raw_target = open(f"../finetuned_results/output_harfbuzz/0{harfbuzz_count}.rawoutput", "w+")
  prompt = generate_and_tokenize_prompt(i)
  generated_target = generate_rawllama_text(prompt)
  raw_target.write(generated_target)
  print(generated_target)
  raw_target.close()
  harfbuzz_count += 1

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

You are a security testing engineer who wants to write a C++ program to execute all lines in a given function by defining and initializing its parameters in a suitable way before fuzzing the function through <code>LLVMFuzzerTestOneInput</code>.
    Carefully study the function signature and its parameters, then generate a fuzz targets by the following instructions. YOU MUST call the function to fuzz in the solution.
    Try as many variations of possible inputs as possible. Do not use a random number generator such as <code>rand()</code>.
    Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>.
    All variables used MUST be declared and initialized. Carefully make sure that the variable and argument types in your code match and compiles successfully. Add type casts to make types match.
    Do not create new variables with the same names as existing variables. If you write code using <code>goto</code>, you MUST MUST also de

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 353, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


You are a security testing engineer who wants to write a C++ program to execute all lines in a given function by defining and initializing its parameters in a suitable way before fuzzing the function through <code>LLVMFuzzerTestOneInput</code>.
    Carefully study the function signature and its parameters, then generate a fuzz targets by the following instructions. YOU MUST call the function to fuzz in the solution.
    Try as many variations of possible inputs as possible. Do not use a random number generator such as <code>rand()</code>.
    Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>.
    All variables used MUST be declared and initialized. Carefully make sure that the variable and argument types in your code match and compiles successfully. Add type casts to make types match.
    Do not create new variables with the same names as existing variables. If you write code using <code>goto</code>, you MUST MUST also de

...

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 346, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


You are a security testing engineer who wants to write a C++ program to execute all lines in a given function by defining and initializing its parameters in a suitable way before fuzzing the function through <code>LLVMFuzzerTestOneInput</code>.
    Carefully study the function signature and its parameters, then generate a fuzz targets by the following instructions. YOU MUST call the function to fuzz in the solution.
    Try as many variations of possible inputs as possible. Do not use a random number generator such as <code>rand()</code>.
    Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>.
    All variables used MUST be declared and initialized. Carefully make sure that the variable and argument types in your code match and compiles successfully. Add type casts to make types match.
    Do not create new variables with the same names as existing variables. If you write code using <code>goto</code>, you MUST MUST also de

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 313, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


You are a security testing engineer who wants to write a C++ program to execute all lines in a given function by defining and initializing its parameters in a suitable way before fuzzing the function through <code>LLVMFuzzerTestOneInput</code>.
    Carefully study the function signature and its parameters, then generate a fuzz targets by the following instructions. YOU MUST call the function to fuzz in the solution.
    Try as many variations of possible inputs as possible. Do not use a random number generator such as <code>rand()</code>.
    Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>.
    All variables used MUST be declared and initialized. Carefully make sure that the variable and argument types in your code match and compiles successfully. Add type casts to make types match.
    Do not create new variables with the same names as existing variables. If you write code using <code>goto</code>, you MUST MUST also de

In [21]:
def generate_contextualiedLlama_text(prompt):
  DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
  peft_model_id = "sallywww/LLaMA_Context_fuzzTargets"
  model = LlamaForCausalLM.from_pretrained(
            "baffo32/decapoda-research-llama-7B-hf",
            load_in_8bit=True,
            torch_dtype=torch.float16,
            device_map= "auto", #comment this line out if encountering cuda OOM or wrong distribution
                                #across GPU and CPU.
        )
  tokenizer = LlamaTokenizer.from_pretrained(
            "baffo32/decapoda-research-llama-7B-hf",
        )
  peftmodel = PeftModel.from_pretrained(model, peft_model_id, torch_dtype=torch.float16)
  inputs = tokenizer(prompt, return_tensors="pt")
  model.config.use_cache = False
  old_state_dict = model.state_dict
  model.state_dict = (lambda self, *_, **__: get_peft_model_state_dict(self, old_state_dict())).__get__(
        model, type(model)
    )
  if torch.__version__ >= "2" and sys.platform != "win32":
        print("compiling the model")
        model = torch.compile(model)
       # input_ids = inputs["input_ids"].to(model.device)
  generation_config = GenerationConfig(
        temperature=0.1,
        top_p=0.75,
        repetition_penalty=1.1,
    )
  with torch.no_grad():
      outputs = peftmodel.generate(input_ids=inputs["input_ids"].cuda(), generation_config=generation_config)
      output_text = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
      print(output_text)
  return output_text

In [23]:
def contextualized_prompt(func_signature):
    full_prompt =f"""<task>
Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>. It is important that the provided solution compiles and actually calls the function specified by the function header:
<function header>
{func_signature}
<function header>
</task>
Please fill in the solution below:
</solution>

</solution>
    """
    return full_prompt

In [24]:
if os.path.isdir("../finetuned_results") is False:
  os.mkdir("../finetuned_results")
if os.path.isdir("../finetuned_results/output_harfbuzz") is False:
  os.mkdir("../finetuned_results/output_harfbuzz")
for i in harfbuzz_sigs:
  raw_target = open(f"../finetuned_results/output_harfbuzz/0{harfbuzz_count}.rawoutput", "w+")
  prompt = contextualized_prompt(i)
  generated_target = generate_contextualiedLlama_text(prompt)
  raw_target.write(generated_target)
  print(generated_target)
  raw_target.close()
  harfbuzz_count += 1

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

compiling the model


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 118, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


<task>
Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>. It is important that the provided solution compiles and actually calls the function specified by the function header:
<function header>
struct hb_face_t * hb_subset_preprocess(struct hb_face_t * source)
<function header>
</task>
Please fill in the solution below:
</solution>

</solution>
    

<task>
Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>. It is important that the provided solution compiles and actually calls the function specified by the function header:
<function header>
struct hb_face_t * hb_subset_preprocess(struct hb_face_t * source)
<function header>
</task>
Please fill in the solution below:
</solution>

</solution>
    



Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

...

<task>
Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>. It is important that the provided solution compiles and actually calls the function specified by the function header:
<function header>
double solve_itp<hb_shape_justify::$_3>(anon * f, double a, double b, double epsilon, double min_y, double max_y, double * ya, double * yb, double * y)
<function header>
</task>
Please fill in the solution below:
</solution>

</solution>
    

<task>
Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>. It is important that the provided solution compiles and actually calls the function specified by the function header:
<function header>
double solve_itp<hb_shape_justify::$_3>(anon * f, double a, double b, double epsilon, double min_y, double max_y, double * ya, double * yb, double * y)
<function header>
</task>
Please fill in the solution below:
</solution>

</solution>


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

compiling the model


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 113, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


<task>
Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>. It is important that the provided solution compiles and actually calls the function specified by the function header:
<function header>
double hb_shape_justify::$_3::operator(anon * this, double x)
<function header>
</task>
Please fill in the solution below:
</solution>

</solution>
    

<task>
Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>. It is important that the provided solution compiles and actually calls the function specified by the function header:
<function header>
double hb_shape_justify::$_3::operator(anon * this, double x)
<function header>
</task>
Please fill in the solution below:
</solution>

</solution>
    



In [45]:
def generate_starcoder_text(prompt):
  DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
  peft_model_id = "sallywww/StarCoder_oneStep_FuzzTargets"
  model = AutoModelForCausalLM.from_pretrained(
            "bigcode/starcoder",
            load_in_8bit=True,
            torch_dtype=torch.float16,
            device_map= "auto", #comment this line out if encountering cuda OOM or wrong distribution
                                #across GPU and CPU.
        )
  tokenizer = AutoTokenizer.from_pretrained(
            "bigcode/starcoder",
        )
  peftmodel = PeftModel.from_pretrained(model, peft_model_id, torch_dtype=torch.float16)
  inputs = tokenizer(prompt, return_tensors="pt")
  model.config.use_cache = False
  old_state_dict = model.state_dict
  model.state_dict = (lambda self, *_, **__: get_peft_model_state_dict(self, old_state_dict())).__get__(
        model, type(model)
    )
  if torch.__version__ >= "2" and sys.platform != "win32":
        print("compiling the model")
        model = torch.compile(model)
       # input_ids = inputs["input_ids"].to(model.device)
  generation_config = PeftConfig.from_pretrained(peft_model_id)
  with torch.no_grad():
      outputs = peftmodel.generate(input_ids=inputs["input_ids"].cuda(), generation_config=generation_config)
      output_text = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
      print(output_text)
  return output_text

In [30]:
import os
if os.path.isdir("../finetuned_results") is False:
  os.mkdir("../finetuned_results")
if os.path.isdir("../finetuned_results/output_harfbuzz") is False:
  os.mkdir("../finetuned_results/output_harfbuzz")
for i in harfbuzz_sigs:
  raw_target = open(f"../finetuned_results/output_harfbuzz/0{harfbuzz_count}.rawoutput", "w+")
  prompt = generate_and_tokenize_prompt(i)
  generated_target = generate_starcoder_text(prompt)
  raw_target.write(generated_target)
  print(generated_target)
  raw_target.close()
  harfbuzz_count += 1

pytorch_model-00001-of-00007.bin:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

pytorch_model-00002-of-00007.bin:   0%|          | 0.00/9.86G [00:00<?, ?B/s]

pytorch_model-00003-of-00007.bin:   0%|          | 0.00/9.85G [00:00<?, ?B/s]

pytorch_model-00004-of-00007.bin:   0%|          | 0.00/9.86G [00:00<?, ?B/s]

pytorch_model-00005-of-00007.bin:   0%|          | 0.00/9.85G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/532 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

SafetensorError: Error while deserializing header: InvalidHeaderDeserialization

In [14]:
def generate_codellama_text(prompt):
  DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
  peft_model_id = "sallywww/codeLLaMA_oneStep_fuzzTargets"
  base_model = "codellama/CodeLlama-7b-hf"
  model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto", #might require commenting out
    )
  model.load_adapter(peft_model_id)
  tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")

  inputs = tokenizer(prompt, return_tensors="pt")
  with torch.no_grad():
      outputs = model.generate(input_ids=inputs["input_ids"].cuda())
      output_text = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
      print(output_text)
  return output_text

In [15]:
if os.path.isdir("../finetuned_results") is False:
  os.mkdir("../finetuned_results")
if os.path.isdir("../finetuned_results/output_harfbuzz") is False:
  os.mkdir("../finetuned_results/output_harfbuzz")
for i in harfbuzz_sigs:
  raw_target = open(f"../finetuned_results/output_harfbuzz/0{harfbuzz_count}.rawoutput", "w+")
  prompt = generate_and_tokenize_prompt(i)
  generated_target = generate_codellama_text(prompt)
  raw_target.write(generated_target)
  print(generated_target)
  raw_target.close()
  harfbuzz_count += 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

SafetensorError: Error while deserializing header: InvalidHeaderDeserialization

In [40]:
def context_tokenize_and_train_llama(
    training_data,
    max_seq_length,
    micro_batch_size,
    gradient_accumulation_steps,
    epochs,
    learning_rate,
    lora_r,
    lora_alpha,
    lora_dropout,
    model_name,
    size=None):

    model = LlamaForCausalLM.from_pretrained(
            "baffo32/decapoda-research-llama-7B-hf",
            load_in_8bit=True,
            torch_dtype=torch.float16,
            device_map= "auto", #comment this line out if encountering cuda OOM or wrong distribution
                                #across GPU and CPU.
        )
    max_memory = get_balanced_memory(
        model,
        max_memory=None,
        no_split_module_classes=["DecoderLayer", "Attention", "MLP", "LayerNorm", "Linear"],
        dtype='float16',
        low_zero=False,
        )

    device_map = infer_auto_device_map(
        model,
        max_memory=max_memory,
        no_split_module_classes=["DecoderLayer", "Attention", "MLP", "LayerNorm", "Linear"],
        dtype='float16'
        )

    tokenizer = LlamaTokenizer.from_pretrained(
            "baffo32/decapoda-research-llama-7B-hf",
        )
    tokenizer.pad_token_id = 0
    training_data = open(training_data, "r").read()
    paragraphs = training_data.split("<end of text>")
    print("Number of samples: " + str(len(paragraphs)))

    def tokenize(item):
        result = tokenizer(
            item["text"],
            truncation=True,
            max_length=max_seq_length,
            padding="max_length",
        )
        return {
            "input_ids": result["input_ids"][:-1],
            "attention_mask": result["attention_mask"][:-1],
        }

    def to_dict(text):
        return {"text": text}

    paragraphs = [to_dict(x) for x in paragraphs]
    data = Dataset.from_list(paragraphs)
    tokenized_train_dataset = data.shuffle().map(lambda x: tokenize(x))
    data = Dataset.from_list(paragraphs)
    data = data.shuffle().map(lambda x: tokenize(x))
    model = prepare_model_for_int8_training(model)
    model = get_peft_model(model, LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=lora_dropout,
        bias="none",
        task_type="CAUSAL_LM",
    ))
    model = model.to("cuda")
    output_dir = f"context-lora-{model_name}"
    print("Training...")

    training_args = transformers.TrainingArguments(
        per_device_train_batch_size=micro_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        num_train_epochs=epochs,
        learning_rate=learning_rate,
        fp16=True,
        logging_steps=20,
        output_dir=output_dir,
        save_total_limit=100,
    )


    trainer = transformers.Trainer(
        model=model,
        train_dataset=tokenized_train_dataset,
        args=training_args,
        data_collator=transformers.DataCollatorForLanguageModeling(
            tokenizer,
            mlm=False,
        ),
    )
    result = trainer.train(resume_from_checkpoint=False)

    model.save_pretrained(output_dir)

    return result

In [41]:
context_tokenize_and_train_llama(
          training_data="/content/llmfz/train_data/smalldata.txt",
          max_seq_length=4096,
          micro_batch_size=1,
          gradient_accumulation_steps=16,
          epochs=30,
          learning_rate=2e-5,
          lora_r=8,
          lora_alpha=32,
          lora_dropout=0.1,
          model_name='lora_llama_light')

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

Number of samples: 15


Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:143: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Training...


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
20,0.766800


TrainOutput(global_step=30, training_loss=0.7336568514506022, metrics={'train_runtime': 1606.5391, 'train_samples_per_second': 0.28, 'train_steps_per_second': 0.019, 'total_flos': 3.8986916806656e+16, 'train_loss': 0.7336568514506022, 'epoch': 16.0})

In [52]:
def generate_lightllama_text(prompt):
  DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

  peft_model_id = "/content/context-lora-lora_llama_light"
  config = PeftConfig.from_pretrained(peft_model_id)
  '''
  model = LlamaForCausalLM.from_pretrained(
            "baffo32/decapoda-research-llama-7B-hf",
            load_in_8bit=True,
            torch_dtype=torch.float16,
            device_map= "auto", #comment this line out if encountering cuda OOM or wrong distribution
                                #across GPU and CPU.
        )
  tokenizer = LlamaTokenizer.from_pretrained(
            "baffo32/decapoda-research-llama-7B-hf",
        )
  peftmodel = PeftModel.from_pretrained(model, peft_model_id, torch_dtype=torch.float16)
  '''
  model = LlamaForCausalLM.from_pretrained(config.base_model_name_or_path)
  model = PeftModel.from_pretrained(model, peft_model_id)
  tokenizer = LlamaTokenizer.from_pretrained(config.base_model_name_or_path)
  model.cuda()
  inputs = tokenizer(prompt, return_tensors="pt")
  model.config.use_cache = False
  old_state_dict = model.state_dict
  model.state_dict = (lambda self, *_, **__: get_peft_model_state_dict(self, old_state_dict())).__get__(
        model, type(model)
    )
  if torch.__version__ >= "2" and sys.platform != "win32":
        print("compiling the model")
        model = torch.compile(model)
       # input_ids = inputs["input_ids"].to(model.device)
  with torch.no_grad():
      outputs = model.generate(input_ids=inputs["input_ids"].cuda())
      output_text = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
      print(output_text)
  return output_text

In [43]:
def generate_target_prompt(func_signature):
    full_prompt =f"""<task>
Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>. It is important that the provided solution compiles and actually calls the function specified by the function header:
<function header>
{func_signature}
</function header>
</task>

    """
    return full_prompt

In [44]:
if os.path.isdir("../finetuned_results") is False:
  os.mkdir("../finetuned_results")
if os.path.isdir("../finetuned_results/output_harfbuzz") is False:
  os.mkdir("../finetuned_results/output_harfbuzz")
for i in harfbuzz_sigs:
  raw_target = open(f"../finetuned_results/output_harfbuzz/0{harfbuzz_count}.rawoutput", "w+")
  prompt = generate_target_prompt(i)
  generated_target = generate_lightllama_text(prompt)
  raw_target.write(generated_target)
  print(generated_target)
  raw_target.close()
  harfbuzz_count += 1

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

compiling the model


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.75` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configurati

<task>
Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>. It is important that the provided solution compiles and actually calls the function specified by the function header:
<function header>
struct hb_face_t * hb_subset_preprocess(struct hb_face_t * source)
</function header>
</task>

     </
<task>
Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>. It is important that the provided solution compiles and actually calls the function specified by the function header:
<function header>
struct hb_face_t * hb_subset_preprocess(struct hb_face_t * source)
</function header>
</task>

     </


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

...

<task>
Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>. It is important that the provided solution compiles and actually calls the function specified by the function header:
<function header>
double solve_itp<hb_shape_justify::$_3>(anon * f, double a, double b, double epsilon, double min_y, double max_y, double * ya, double * yb, double * y)
</function header>
</task>

     </
<task>
Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>. It is important that the provided solution compiles and actually calls the function specified by the function header:
<function header>
double solve_itp<hb_shape_justify::$_3>(anon * f, double a, double b, double epsilon, double min_y, double max_y, double * ya, double * yb, double * y)
</function header>
</task>

     </


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

compiling the model


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 95, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


<task>
Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>. It is important that the provided solution compiles and actually calls the function specified by the function header:
<function header>
double hb_shape_justify::$_3::operator(anon * this, double x)
</function header>
</task>

     </
<task>
Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>. It is important that the provided solution compiles and actually calls the function specified by the function header:
<function header>
double hb_shape_justify::$_3::operator(anon * this, double x)
</function header>
</task>

     </


In [53]:
if os.path.isdir("../finetuned_results") is False:
  os.mkdir("../finetuned_results")
if os.path.isdir("../finetuned_results/output_harfbuzz") is False:
  os.mkdir("../finetuned_results/output_harfbuzz")
for i in harfbuzz_sigs:
  raw_target = open(f"../finetuned_results/output_harfbuzz/0{harfbuzz_count}.rawoutput", "w+")
  prompt = generate_target_prompt(i)
  generated_target = generate_lightllama_text(prompt)
  raw_target.write(generated_target)
  print(generated_target)
  raw_target.close()
  harfbuzz_count += 1

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

compiling the model
<task>
Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>. It is important that the provided solution compiles and actually calls the function specified by the function header:
<function header>
struct hb_face_t * hb_subset_preprocess(struct hb_face_t * source)
</function header>
</task>

    

<task>
Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>. It is important that the provided solution compiles and actually calls the function specified by the function header:
<function header>
struct hb_face_t * hb_subset_preprocess(struct hb_face_t * source)
</function header>
</task>

    



Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

compiling the model
<task>
Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>. It is important that the provided solution compiles and actually calls the function specified by the function header:
<function header>
void hb_ot_shape_glyphs_closure(struct hb_font_t * font, struct hb_buffer_t * buffer, struct hb_feature_t * features, int num_features, struct hb_set_t * glyphs)
</function header>
</task>

    

<task>
Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>. It is important that the provided solution compiles and actually calls the function specified by the function header:
<function header>
void hb_ot_shape_glyphs_closure(struct hb_font_t * font, struct hb_buffer_t * buffer, struct hb_feature_t * features, int num_features, struct hb_set_t * glyphs)
</function header>
</task>

    



/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 135, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


...

compiling the model
<task>
Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>. It is important that the provided solution compiles and actually calls the function specified by the function header:
<function header>
double solve_itp<hb_shape_justify::$_3>(anon * f, double a, double b, double epsilon, double min_y, double max_y, double * ya, double * yb, double * y)
</function header>
</task>

     <
<task>
Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>. It is important that the provided solution compiles and actually calls the function specified by the function header:
<function header>
double solve_itp<hb_shape_justify::$_3>(anon * f, double a, double b, double epsilon, double min_y, double max_y, double * ya, double * yb, double * y)
</function header>
</task>

     <


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 128, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

compiling the model
<task>
Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>. It is important that the provided solution compiles and actually calls the function specified by the function header:
<function header>
double hb_shape_justify::$_3::operator(anon * this, double x)
</function header>
</task>

    

<task>
Your goal is to write a fuzzing harness for the provided function header using <code>LLVMFuzzerTestOneInput</code>. It is important that the provided solution compiles and actually calls the function specified by the function header:
<function header>
double hb_shape_justify::$_3::operator(anon * this, double x)
</function header>
</task>

    



/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 95, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
